# How to Culculate Forward Outright Price

This article will demonstrate how to use Refinitiv Data Platfrom(RDP) libraries to calculate Forward Outright Price. Forward Outright Price is the predicted metal price based on current price and forward rate.

# Introduction to the RDP Libraries

[Refinitiv Data Platform (RDP)](https://developers.refinitiv.com/refinitiv-data-platform). is our cloud-enabled, open platform, that brings together content, analytics, customer and third-party data distribution and management technology in one place.Hence, it would be ideal if a single library could be used to access that content which is in once place as well. That’s why [Refinitiv Data Platform Libraries](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries) has been built. The libraries make simplify integration into all delivery platforms which are Refinitiv Workspace, or directly to RDP or Elektron. The libraries provide a consistent API experience. You just learn once and apply across the wealth of Refinitiv content. The libraries reach the largest audience of developers possible from Citizen developers to seasoned professionals. For more details, please refer to [Refinitiv Data Platform Libraries - An Introduction](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/docs?content=62446&type=documentation_item)

# Forward Outright Price Calculation

The formula to calculate Forward Outright Price is the below:

 <H2>Forward Outright Price = Spot Price + Forward Swap Price</H2>

- **Spot Price** = snapshot real-time price e.g. Bid and Ask

- **Forward Swap Price** = Forward Swap Point / 100

- **Forward Swap Point** = (Spot Price * (Forward Rate * days)) / Yearly basis

Special case on Overnight (ON) and Tomorrow Next (TN) Forward Outright Price, switch Bid/Ask and change sign of Forward Swap Price:
- ON Forward Outright Bid Price = Bid Spot Price + (-TN Forward Ask Swap Price) + (-ON Forward Ask Swap Price)
- ON Forward Outright Ask Price = Bid Spot Price + (-TN Forward Bid Swap Price) + (-ON Forward Bid Swap Price)
- TN Forward Outright Bid Price = Bid Spot Price + (-TN Forward Ask Swap Price)
- TN Forward Outright Ask Price = Ask Spot Price + (-TN Forward Bid Swap Price)

# Prerequisite
1. Access Credentials. 
<br>To be able to access data used to calculate Forward Outright Price, you need to have the access credentials for **Platform - Refinitiv Data Platform** end point. Please refer to [Access Credentials Quick Start](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start) for process and details how to obtain the access credentials for this end point.
2. Python environment and package installer.
   * [Anaconda](https://www.anaconda.com/distribution/). It is the easiest way to perform Python data science and machine learning on Linux, Windows, and Mac OS X. It consists of Jupyter Notebook and the packages that we will require e.g. numpy, pandas. You can download and install Anaconda from [here.](https://www.anaconda.com/distribution/)
   * [cufflinks.](https://plotly.com/python/cufflinks/)(Optional to plot graphs) When you import cufflinks, all Pandas data frames and series objects have a new method attached to them called .iplot(). Hence, the Pandas data frames can plots the graph. To install cufflinks: 
        1. Open Anaconda Prompt:
        2. Run the following command:
```
pip install cufflinks
```
3. RDP Library for Python. To install the library:
   1. Open Anaconda Prompt:
   2. Run the following command:
```
pip install refinitiv.dataplatform 
```

## Implementation

### Import the libraries

In [1]:
import refinitiv.dataplatform as rdp
import getpass
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta, timezone
import sys
import cufflinks as cf

### Create the logger

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### Define the functions

Getting snapshot real-time data using RDP library function

In [3]:
def get_rt(instrument, fids):
    logger.info(f'Real-time data of {instrument} retrieving')
    # get snapshot real-time data using RDP library function
    instrument_rt = rdp.get_snapshot(instrument, fids)
    get_snapshot_dt_utc = datetime.now(tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
    # if getting data fails
    if instrument_rt is None: 
        if rdp.get_last_status()['error']: 
            logger.error(f'Getting real-time data failed with the error: ' + str(rdp.get_last_status()['error'])) 
        else:
            logger.error(f'Getting real-time data failed') 
        sys.exit(1)
    # if getting data succeeds
    else:
        instrument_rt.insert(loc=0, column='DATE', value=datetime.now(tz=timezone.utc).date())
        instrument_rt.loc[:, 'DATE'] = pd.to_datetime(instrument_rt.loc[:, 'DATE'], format='%Y-%m-%d')
        return instrument_rt, get_snapshot_dt_utc

Getting data using Delivery library layer function which sends REST Request to RDP service  

In [4]:
def rdp_url_request(session, url, method, query_parameters=None, body_parameters=None, err_detail=None):
    if query_parameters:
        logger.info(f'RDP calling with {url}: {query_parameters} ')
    if body_parameters:
        logger.info(f'RDP calling with {url}: {body_parameters}')
    #create end point to RDP service with the session 
    endpoint = rdp.DeliveryFactory.create_end_point(session=session, url=url)
    #send the request to RDP service and wait for response
    response = endpoint.send_request(method=method, query_parameters=query_parameters, body_parameters=body_parameters)
    #print if request is successful
    logger.info(f'RDP response is success: {response.is_success}')
    error = False
    #if request is unsuccessful
    if response.error_code:
        logger.error(f'RDP response error code: {response.error_code}')
        error = True
    if response.error_message:
        logger.error(f'RDP response error message: {response.error_message}')
        error = True
    strRaw = str(response.data.raw)
    if strRaw.find("Error") >= 0:
        logger.error(f"The RDP response contains the error:" + strRaw)
        error = True
    #if request is not successful, exit
    if error:
        if err_detail is not None:
            logger.error(err_detail)
        sys.exit(1)
    #if request is successful, return response
    return response

Getting underlying RICs of the chain RIC function

In [5]:
def get_chain(session, instrument):
    try:
        #specify a chain RIC
        query_parameters = {'universe': instrument}
        #request underlying RICs from chains RDP service
        chain_response = rdp_url_request(session, url='data/pricing/beta3/views/chains',
                                     method=rdp.Endpoint.RequestMethod.GET, query_parameters=query_parameters,
                                         err_detail='The function cannot access chain.')
        #get raw response
        result = chain_response.data.raw
        #getting underlying RICs from the raw response
        r_list = result['data']['constituents']
        logger.info(f'RDP chain response: {r_list}')
        return r_list
    except Exception as e:
        logger.exception('The function cannot access chain. The exception is:')
        logger.exception(e)
        sys.exit(1)  

Getting trade dates function which sends a request to **date-schedule** API endpoint to get date schedules. For more details of this API endpoint, please refer to [Quantitative Analytics - Dates and Calendars in API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs#/details/L2RhdGEvcXVhbnRpdGF0aXZlLWFuYWx5dGljcy1kYXRlcy1hbmQtY2FsZW5kYXJzL2JldGEx/L2RhdGUtc2NoZWR1bGU=/POST/SWAGGER)

**Quantitative Analytics - Dates and Calendars** is a part of **Quantitative Pricing Service(QPS)** on RDP. QPS performs financial calculation on provided data. This includes contract pricing, curve and surface calibration and interpolation, calendar-related operations, and more generally operations that rely on Adfin or PriceIt calculation libraries.

In [6]:
def qps_gen_tradedate(date, calendar_ref, session):
    try:
        date = date + 'T00:00:00.000Z'
        body = {'start': date,
                'count': 10,
                'frequency': 'Daily',
                'rollConventionType': 'CalendarDayOfMonth',
                'dateAdjustment': {'adjustmentDirection': 'Before',
                                   'dateMovingConvention': 'ModifiedFollowing',
                                   'endOfMonthConvention': 'Same',
                                   'calendars': [calendar_ref]
                                   }
                }
        r = rdp_url_request(session, url='data/quantitative-analytics-dates-and-calendars/beta1/date-schedule',
                            method=rdp.Endpoint.RequestMethod.POST, body_parameters=body, 
                            err_detail='The function cannot access trading day')
        r1 = r.data.raw
        r2 = []
        for _, value in r1.items():
            value
            r2.append(value)
        r2 = pd.DataFrame(r2)
        r2 = r2.T
        r2.columns = ['DATE']
        r2.loc[:, 'DATE'] = r2.loc[:, 'DATE'].str[:10]
        r2.loc[:, 'DATE'] = pd.to_datetime(
            r2.loc[:, 'DATE'], format='%Y-%m-%d')
        return r2
    except Exception as e:
        logger.exception('The function cannot access trading day. The exception is:')
        logger.exception(e)
        sys.exit(1)

Getting trade dates function which excludes holidays of calendar reference e.g. USA holidays

In [7]:
def skip_holiday(df, column, calendar_ref, session):
    try:
        date = df.iloc[0]['DATE']
        date = date.strftime('%Y-%m-%d')
        df_trade_date = qps_gen_tradedate(date, calendar_ref, session)
        df = df.reset_index(drop=True)
        r = []
        for sted in df[column]:
            for td in df_trade_date.loc[:, 'DATE']:
                if sted not in df_trade_date.loc[:, 'DATE']:
                    if td >= sted:
                        sted = td
                        r.append(sted)
                        break
                else:
                    r.append(sted)
                    break
        r = pd.DataFrame(r)
        df.loc[:, column] = r[0]
        return df
    except Exception as e:
        logger.exception('The function cannot skip holiday. The exception is:')
        logger.exception(e)
        sys.exit(1)

Calculating start date function. The function adds *START* column to the dataframe then returns the dataframe. The *START* column is trade dates based on TENOR, transaction date, calendar reference for skipping holidays. 

[TENOR](https://www.investopedia.com/terms/t/tenor.asp) refers to the length of time remaining before a financial contract expires[1].

[Transaction date](https://www.investopedia.com/terms/t/transaction-date.asp) is a date upon which a trade takes place for a security or other financial instrument[2].

In [8]:
def cal_startdate(df, transaction_date, calendar_ref, session):
    try:
        transaction_date = int(transaction_date)
        date1 = df.iloc[0]['DATE']
        df.loc[:, 'START'] = date1
        # ON(Overnight) condition
        df.loc[df['TENOR'] == 'ON', 'START'] = date1
        # TN(Tomorrow next) condition
        df.loc[df['TENOR'] == 'TN', 'START'] = date1 + timedelta(days=1)
        df.loc[((df['TENOR'] == 'TN') & (df['START'].dt.dayofweek == 5)),
               'START'] = date1 + timedelta(days=3)
        df.loc[((df['TENOR'] == 'TN') & (df['START'].dt.dayofweek == 6)),
               'START'] = date1 + timedelta(days=2)
        df.loc[(df['TENOR'] == 'ON') & (df['TENOR'] == 'TN') & (
            df['START'].dt.dayofweek == 6), 'START'] = df['START'] + timedelta(days=3)
        # other tenors condition
        df.loc[(df['TENOR'] != 'ON') & (df['TENOR'] != 'TN'),
               'START'] = date1 + timedelta(days=transaction_date)
        df.loc[(df['TENOR'] != 'ON') & (df['TENOR'] != 'TN') & (
            df['START'].dt.dayofweek >= 5), 'START'] = df['START'] + timedelta(days=transaction_date)
        if df.iloc[0]['TENOR'] == 'ON':
            dfontn = df.iloc[0:2]
            df2 = df.iloc[2:]
            df3 = skip_holiday(df2, 'START', calendar_ref, session)
            df = dfontn.append(df3, ignore_index=True)
        else:
            df = skip_holiday(df, 'START', calendar_ref, session)
        return df
    except Exception as e:
        logger.exception('The function cannot calculate start date. The exception is:')
        logger.exception(e)
        sys.exit(1)

Calculating end date function. The function adds *END* column to the dataframe then returns the dataframe. The *END* column is trade dates based on TENOR, calendar reference for skipping holidays. 

In [9]:
def qps_cal_enddate(df, calendar_ref, session):
    try:
        on_stdate = df.loc[df['TENOR'] == 'ON', 'START']
        tn_stdate = df.loc[df['TENOR'] == 'TN', 'START']
        df.loc[:, 'START'] = df['START'].dt.strftime('%Y-%m-%d')
        df.loc[:, 'START'] = df['START'] + 'T00:00:00.000Z'
        df2 = df[['START', 'TENOR']].copy()
        body = []
        for date, tenor in df2.itertuples(index=False):
            item = {
                'startDate': date,
                'period': tenor,
                'calendars': [calendar_ref]
                #'calendars': []
            }
            body.append(item)
        r = rdp_url_request(session, url='data/quantitative-analytics-dates-and-calendars/beta1/AddPeriods',
                            method=rdp.Endpoint.RequestMethod.POST, body_parameters=body,
                            err_detail='The function cannot calculate end date by QPS')
        r1 = r.data.raw
        r2 = pd.DataFrame(r1)
        r2.loc[:, 'date'] = r2['date'].str[:10]
        df.loc[:, 'END'] = r2['date']
        df.loc[:, 'START'] = df['START'].str[:10]
        df.loc[:, 'START'] = pd.to_datetime(df['START'], format='%Y-%m-%d')
        df.loc[df['TENOR'] == 'ON', 'END'] = on_stdate + timedelta(days=1)
        df.loc[df['TENOR'] == 'TN', 'END'] = tn_stdate + timedelta(days=1)
        df.loc[:, 'END'] = pd.to_datetime(df['END'], format='%Y-%m-%d')
        df.loc[(df['TENOR'] == 'ON') & (df['END'].dt.dayofweek == 6),
               'END'] = on_stdate + timedelta(days=2)
        df.loc[(df['TENOR'] == 'ON') & (df['END'].dt.dayofweek == 0),
               'END'] = on_stdate + timedelta(days=1)
        df.loc[(df['TENOR'] == 'TN') & (df['END'].dt.dayofweek == 6),
               'END'] = tn_stdate + timedelta(days=2)
        df.loc[(df['TENOR'] == 'TN') & (df['END'].dt.dayofweek == 0),
               'END'] = tn_stdate + timedelta(days=1)
        if df.iloc[0]['TENOR'] == 'ON':
            df2 = df.iloc[0:2]
            df3 = skip_holiday(df2, 'END', calendar_ref, session)
            df.iloc[0:2] = df3
        return df
    except Exception as e:
        logger.exception('The function cannot calculate end date by QPS. The exception is:')
        logger.exception(e)
        sys.exit(1)

Creating interpolation dataframe function based on defined TENOR list and the current date(UTC timezone). The function returns dataframe containing DATE and TENOR column

In [10]:
def create_df_interp(tenorlist):
    try:
        logger.info('Rate interpolating')
        df_interp = pd.DataFrame(tenorlist, columns=['TENOR'])
        df_interp.loc[:, 'DATE'] = datetime.now(tz=timezone.utc).date()
        df_interp.loc[:, 'DATE'] = pd.to_datetime(
            df_interp['DATE'], format='%Y-%m-%d')
        df_interp = df_interp[['DATE', 'TENOR']].copy()
        return df_interp
    except Exception as e:
        logger.exception('The function cannot create interpolation dataframe. The exception is:')
        logger.exception(e)
        sys.exit(1)

The function returns the list of days count from the date 01/01/01 based on the date series input

In [11]:
def datelist_toordinal(df):
    try:
        ii = []
        for i in df:
            i = i.toordinal()
            ii.append(i)
        return ii
    except Exception as e:
        logger.exception('The function cannot change the data to ordinal number. The exception is:')
        logger.exception(e)
        sys.exit(1)

The funtion calculates RATE of each TENOR and returns the dataframe containing DATE,TENOR,START,END and RATE column

In [12]:
def cal_interp(df_ref, df_interp):
    try:
        del df_ref['Instruments']
        df_interp = df_interp.reset_index().merge(
            df_ref, how='left').set_index('index')
        dftemp = df_interp[['END', 'RATE']].copy()
        dftemp.loc[:, 'END'] = datelist_toordinal(dftemp['END'])
        df_ref.loc[:, 'END'] = datelist_toordinal(df_ref['END'])
        dftemp['RATE'].fillna('x', inplace=True)
        r = []
        for end, rate in dftemp.itertuples(index=False):
            if rate == 'x':
                rate = np.interp(end, df_ref['END'], df_ref['RATE'])
                r.append(rate)
            else:
                r.append(rate)
        df_interp.loc[:, 'RATE'] = r
        return df_interp
    except Exception as e:
        logger.exception('The function cannot interpolate the data. The exception is:')
        logger.exception(e)
        sys.exit(1)

The function gets current metal spot price(bid and ask) and current date time

In [13]:
def metal_update(metal):
    metal_rt, metal_dt_utc = get_rt([metal], ['BID', 'ASK'])
    bid = metal_rt.iloc[0]['BID']
    ask = metal_rt.iloc[0]['ASK']
    return bid, ask, metal_dt_utc

The function culcaltes forward rate. It returns dataframe containing DATE,TENOR,START,END and RATE(forward rate) column

In [14]:
def rate_update(session, calendar_ref, transaction_date, tenor_list, rate, lending_rate_field, rate_chain, lending_rate_ref_af=None):
    logger.info('Lending rate updating')
    rate_fids = ['GV4_TEXT', lending_rate_field]
    rate_rt, rate_dt_utc = get_rt(rate_chain, rate_fids)
    rate_rt.rename(columns={'GV4_TEXT': 'TENOR',
                            lending_rate_field: 'RATE'}, inplace=True)
    ###
    # Map specific rate
    df_startdate = cal_startdate(rate_rt, transaction_date, calendar_ref, session)
    df_startenddate = qps_cal_enddate(df_startdate, calendar_ref, session)
    # Rate Interpolation
    interp_frame = create_df_interp(tenor_list)
    date_interp_start = cal_startdate(interp_frame, transaction_date, calendar_ref, session)
    if lending_rate_ref_af is not None:
        date_interp_end = lending_rate_ref_af
    else:
        date_interp_end = qps_cal_enddate(date_interp_start, calendar_ref, session)
    rate_interp = cal_interp(df_startenddate, date_interp_end)
    return rate_interp, date_interp_end

The function calculates forward swap points and forward swap prices according to the formula. It returns the dataframe with new column named BID_RATE(rename from RATE column),ASK_RATE,DAYS,BID_SWAP_POINT(BID forward swap point),ASK_SWAP_POINT(ASK forward swap point),BID_SWAP_PRICE(forward swap prices),ASK_SWAP_PRICE(forward swap prices)

In [15]:
def cal_swppoint(df_afterinterp, bid, ask):
    try:
        logger.info('Swap point calculating')
        df_afterinterp = df_afterinterp.rename(columns={'RATE': 'BID_RATE'})
        df_afterinterp.loc[:, 'ASK_RATE'] = df_afterinterp['BID_RATE']
        df_afterinterp.loc[:, 'ASK_RATE'] += proxy_price_spread
        df_afterinterp.loc[:, 'DAYS'] = (df_afterinterp['END'] - df_afterinterp['START']) / timedelta(days=1)
        df_afterinterp.loc[:, 'BID_SWAP_POINT'] = ((df_afterinterp['BID_RATE'].mul(df_afterinterp['DAYS'])).mul(bid)).div(365)
        df_afterinterp.loc[:, 'ASK_SWAP_POINT'] = ((df_afterinterp['ASK_RATE'].mul(df_afterinterp['DAYS'])).mul(ask)).div(365)
        #calculate Forward swap price from Forward swap point
        df_afterinterp.loc[:,'BID_SWAP_PRICE'] = df_afterinterp['BID_SWAP_POINT'].div(100)
        df_afterinterp.loc[:,'ASK_SWAP_PRICE'] = df_afterinterp['ASK_SWAP_POINT'].div(100)
        return df_afterinterp
    except Exception as e:
        logger.exception('The function cannot calculate the swap point. The exception is:')
        logger.exception(e)
        sys.exit(1)

The function calculates forward outright prices according to the formula. It returns the dataframe with forward outright prices which is BID_FWDOUTRIGHT and ASK_FWDOUTRIGHT column

In [16]:
def cal_fwdoutright(df, bid, ask):
    try:
        df.loc[(df['TENOR'] != 'ON') & (df['TENOR'] != 'TN'),'BID_FWDOUTRIGHT'] = bid + df['BID_SWAP_PRICE'] 
        df.loc[(df['TENOR'] != 'ON') & (df['TENOR'] != 'TN'),'ASK_FWDOUTRIGHT'] = ask + df['ASK_SWAP_PRICE'] 
        bid_on = df.loc[(df['TENOR'] == 'ON'), 'BID_SWAP_PRICE']
        bid_on = bid_on[0]
        bid_on *= -1
        ask_on = df.loc[(df['TENOR'] == 'ON'), 'ASK_SWAP_PRICE']
        ask_on = ask_on[0]
        ask_on *= -1
        bid_tn = df.loc[(df['TENOR'] == 'TN'), 'BID_SWAP_PRICE']
        bid_tn = bid_tn[1]
        bid_tn *= -1
        ask_tn = df.loc[(df['TENOR'] == 'TN'), 'ASK_SWAP_PRICE']
        ask_tn = ask_tn[1]
        ask_tn *= -1
        df.loc[(df['TENOR'] == 'ON'), 'BID_FWDOUTRIGHT'] = bid + ask_tn + ask_on
        df.loc[(df['TENOR'] == 'ON'), 'ASK_FWDOUTRIGHT'] = ask + bid_tn + bid_on
        df.loc[(df['TENOR'] == 'TN'), 'BID_FWDOUTRIGHT'] = bid + ask_tn
        df.loc[(df['TENOR'] == 'TN'), 'ASK_FWDOUTRIGHT'] = ask + bid_tn
        return df
    except Exception as e:
        logger.exception('The function cannot calculate the forward outright. The exception is:')
        logger.exception(e)
        sys.exit(1)

The function calculates forward outright prices based on the forward swap prices, metal spot price(bid and ask)

In [17]:
def calculate_fwdoutright(metal, df_swppt, sp_bid, sp_ask):
    logger.info('Forward outright calculating')
    df_swppoint_cal = df_swppt.copy()
    df_fwdoutright = cal_fwdoutright(df_swppoint_cal, sp_bid, sp_ask)
    return df_fwdoutright

### Main method ###

### Step 1: Opening a session

To Open a session to RDP directly. RDP username, password and the application key are required.

In [18]:
RDP_USER = input("Enter RDP username: ")
RDP_PASSWORD = getpass.getpass('Enter RDP password:')
APP_KEY = getpass.getpass("Enter the app key: ")

Enter RDP username: pimchaya.wongrukun1@thomsonreuters.com
Enter RDP password:········
Enter the app key: ········


Create and open a platform session with application key, RDP username and password to connect to RDP directly.

In [19]:
session = rdp.open_platform_session(
    APP_KEY, 
    rdp.GrantPassword(
        username = RDP_USER, 
        password = RDP_PASSWORD
    )
)

### Step 2: Set the required info
Input the required info

In [20]:
metal_input = input("Enter a metal RIC e.g. XPD= : ")
lending_rate_input = input("Enter forward offer rate RIC e.g. XPDFWD=TTKL : ")
lending_rate_field_input = input("Enter forward offer rate Field Name(default is PRIMACT_1): ") 
if len(lending_rate_field_input.strip()) == 0:
    lending_rate_field_input = "PRIMACT_1"
transaction_date_input = input("Enter transaction date(default is 2): ")
if len(transaction_date_input.strip()) == 0:
    transaction_date_input = '2'
calendar_ref_input = input("Enter calendars(default is USA): ")
if len(calendar_ref_input.strip()) == 0:
    calendar_ref_input = 'USA'
proxy_price_spread_input_str = input("Enter proxy price spread(default is 0.08): ")
if len(proxy_price_spread_input_str.strip()) == 0:
    proxy_price_spread_input = 0.08
else:
    try:
        fVal = float(proxy_price_spread_input_str)
    except ValueError:
        proxy_price_spread_input = 0.08
        logger.info("Proxy price spread is not integer or float so the default(0.08) is used.")
    else:
        proxy_price_spread_input = fVal

Enter a metal RIC e.g. XPD= : XPD=
Enter forward offer rate RIC e.g. XPDFWD=TTKL : XPDFWD=TTKL
Enter forward offer rate Field Name(default is PRIMACT_1): 
Enter transaction date(default is 2): 
Enter calendars(default is USA): 
Enter proxy price spread(default is 0.08): 


Set the configuration according to the inputs then assign the configuration to the variables

In [21]:
config = {
        'metal': metal_input,
        'lending_rate': lending_rate_input,
        'lending_rate_field': lending_rate_field_input,
        'calendar_ref': calendar_ref_input,
        'transaction_date': transaction_date_input,
        'proxy_price_spread': proxy_price_spread_input,
         #ON - Overnight, TN - Tomorrow next, SN - Spot next, SW - Spot week
        'tenor_list': [
            'ON', 'TN', 'SN', 'SW', '2W', '3W', '1M', '2M', '3M', '4M', '5M', '6M', '7M', '8M', '9M', '10M', 
            '11M', '1Y', '15M', '18M', '21M', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y'
        ]
}
metal = config['metal']
lending_rate = config['lending_rate']
lending_rate_field = config['lending_rate_field']
calendar_ref = config['calendar_ref']
transaction_date = config['transaction_date']
tenor_list = config['tenor_list']
proxy_price_spread = config['proxy_price_spread']

<a name="Step-3"></a>
### Step 3: Request underlying RICs of lending rate chain metal RIC 

In [22]:
lending_rate_chain = get_chain(session, lending_rate)
print("lending rate chain:")
lending_rate_chain

2020-07-02 18:59:37,026 P[13896] [MainThread 18812] RDP calling with data/pricing/beta3/views/chains: {'universe': 'XPDFWD=TTKL'} 
2020-07-02 18:59:38,947 P[13896] [MainThread 18812] RDP response is success: True
2020-07-02 18:59:38,947 P[13896] [MainThread 18812] RDP chain response: ['XPD1M=TTKL', 'XPD1M=TTKL', 'XPD2M=TTKL', 'XPD3M=TTKL', 'XPD6M=TTKL', 'XPD9M=TTKL', 'XPD1Y=TTKL', 'XPD2Y=TTKL']


lending rate chain:


['XPD1M=TTKL',
 'XPD1M=TTKL',
 'XPD2M=TTKL',
 'XPD3M=TTKL',
 'XPD6M=TTKL',
 'XPD9M=TTKL',
 'XPD1Y=TTKL',
 'XPD2Y=TTKL']

### Step 4: Get the spot price, bid and ask, of metal RIC

In [23]:
# 1 metal update
spot_bid, spot_ask, metal_datetime = metal_update(metal)
print("Request data of RIC " + metal + " at " + metal_datetime + "(UTC)")
print("BID is " + str(spot_bid))
print("ASK is " + str(spot_ask))

2020-07-02 18:59:38,971 P[13896] [MainThread 18812] Real-time data of ['XPD='] retrieving


Request data of RIC XPD= at 2020-07-02 11:59:40(UTC)
BID is 1909.735
ASK is 1934.735


### Step 5: Calculate the forward rate
This is the forward rate based on the TENOR and the rate of RICs got from [Step 3: Request underlying RICs of lending rate chain metal RIC](#Step-3)

In [24]:
# 2 lending rate update
lending_rate_rt, lending_rate_ref_af = rate_update(
    session, calendar_ref, transaction_date, tenor_list, lending_rate, lending_rate_field, lending_rate_chain)
lending_rate_rt

2020-07-02 18:59:40,970 P[13896] [MainThread 18812] Lending rate updating
2020-07-02 18:59:40,971 P[13896] [MainThread 18812] Real-time data of ['XPD1M=TTKL', 'XPD1M=TTKL', 'XPD2M=TTKL', 'XPD3M=TTKL', 'XPD6M=TTKL', 'XPD9M=TTKL', 'XPD1Y=TTKL', 'XPD2Y=TTKL'] retrieving
2020-07-02 18:59:43,110 P[13896] [MainThread 18812] RDP calling with data/quantitative-analytics-dates-and-calendars/beta1/date-schedule: {'start': '2020-07-02T00:00:00.000Z', 'count': 10, 'frequency': 'Daily', 'rollConventionType': 'CalendarDayOfMonth', 'dateAdjustment': {'adjustmentDirection': 'Before', 'dateMovingConvention': 'ModifiedFollowing', 'endOfMonthConvention': 'Same', 'calendars': ['USA']}}
2020-07-02 18:59:44,951 P[13896] [MainThread 18812] RDP response is success: True
2020-07-02 18:59:45,005 P[13896] [MainThread 18812] RDP calling with data/quantitative-analytics-dates-and-calendars/beta1/AddPeriods: [{'startDate': '2020-07-06T00:00:00.000Z', 'period': '1M    ', 'calendars': ['USA']}, {'startDate': '2020-07

,DATE,TENOR,START,END,RATE
index,,,,,
0,2020-07-02,ON,2020-07-02,2020-07-03,-0.380000
1,2020-07-02,TN,2020-07-03,2020-07-06,-0.380000
2,2020-07-02,SN,2020-07-06,2020-07-07,-0.380000
3,2020-07-02,SW,2020-07-06,2020-07-13,-0.380000
4,2020-07-02,2W,2020-07-06,2020-07-20,-0.380000
5,2020-07-02,3W,2020-07-06,2020-07-27,-0.380000
6,2020-07-02,1M,2020-07-06,2020-08-06,-0.380000
7,2020-07-02,2M,2020-07-06,2020-09-08,-0.750000
8,2020-07-02,3M,2020-07-06,2020-10-06,-0.750000


### Step 6: Calculate Forward Swap Points and Forward Swap Prices

In [25]:
df_swppoint = cal_swppoint(lending_rate_rt, spot_bid, spot_ask)
df_swppoint

2020-07-02 18:59:52,784 P[13896] [MainThread 18812] Swap point calculating


,DATE,TENOR,START,END,BID_RATE,ASK_RATE,DAYS,BID_SWAP_POINT,ASK_SWAP_POINT,BID_SWAP_PRICE,ASK_SWAP_PRICE
index,,,,,,,,,,,
0,2020-07-02,ON,2020-07-02,2020-07-03,-0.380000,-0.300000,1.0,-1.988217,-1.590193,-0.019882,-0.015902
1,2020-07-02,TN,2020-07-03,2020-07-06,-0.380000,-0.300000,3.0,-5.964652,-4.770579,-0.059647,-0.047706
2,2020-07-02,SN,2020-07-06,2020-07-07,-0.380000,-0.300000,1.0,-1.988217,-1.590193,-0.019882,-0.015902
3,2020-07-02,SW,2020-07-06,2020-07-13,-0.380000,-0.300000,7.0,-13.917521,-11.131352,-0.139175,-0.111314
4,2020-07-02,2W,2020-07-06,2020-07-20,-0.380000,-0.300000,14.0,-27.835042,-22.262704,-0.278350,-0.222627
5,2020-07-02,3W,2020-07-06,2020-07-27,-0.380000,-0.300000,21.0,-41.752562,-33.394056,-0.417526,-0.333941
6,2020-07-02,1M,2020-07-06,2020-08-06,-0.380000,-0.300000,31.0,-61.634735,-49.295988,-0.616347,-0.492960
7,2020-07-02,2M,2020-07-06,2020-09-08,-0.750000,-0.670000,64.0,-251.143233,-227.291608,-2.511432,-2.272916
8,2020-07-02,3M,2020-07-06,2020-10-06,-0.750000,-0.670000,92.0,-361.018397,-326.731686,-3.610184,-3.267317


### Step 7: Calculate Forward Outright Prices

In [26]:
df_fwd = calculate_fwdoutright(metal, df_swppoint, spot_bid, spot_ask)
df_fwd

2020-07-02 18:59:52,834 P[13896] [MainThread 18812] Forward outright calculating


,DATE,TENOR,START,END,BID_RATE,ASK_RATE,DAYS,BID_SWAP_POINT,ASK_SWAP_POINT,BID_SWAP_PRICE,ASK_SWAP_PRICE,BID_FWDOUTRIGHT,ASK_FWDOUTRIGHT
index,,,,,,,,,,,,,
0,2020-07-02,ON,2020-07-02,2020-07-03,-0.380000,-0.300000,1.0,-1.988217,-1.590193,-0.019882,-0.015902,1909.798608,1934.814529
1,2020-07-02,TN,2020-07-03,2020-07-06,-0.380000,-0.300000,3.0,-5.964652,-4.770579,-0.059647,-0.047706,1909.782706,1934.794647
2,2020-07-02,SN,2020-07-06,2020-07-07,-0.380000,-0.300000,1.0,-1.988217,-1.590193,-0.019882,-0.015902,1909.715118,1934.719098
3,2020-07-02,SW,2020-07-06,2020-07-13,-0.380000,-0.300000,7.0,-13.917521,-11.131352,-0.139175,-0.111314,1909.595825,1934.623686
4,2020-07-02,2W,2020-07-06,2020-07-20,-0.380000,-0.300000,14.0,-27.835042,-22.262704,-0.278350,-0.222627,1909.456650,1934.512373
5,2020-07-02,3W,2020-07-06,2020-07-27,-0.380000,-0.300000,21.0,-41.752562,-33.394056,-0.417526,-0.333941,1909.317474,1934.401059
6,2020-07-02,1M,2020-07-06,2020-08-06,-0.380000,-0.300000,31.0,-61.634735,-49.295988,-0.616347,-0.492960,1909.118653,1934.242040
7,2020-07-02,2M,2020-07-06,2020-09-08,-0.750000,-0.670000,64.0,-251.143233,-227.291608,-2.511432,-2.272916,1907.223568,1932.462084
8,2020-07-02,3M,2020-07-06,2020-10-06,-0.750000,-0.670000,92.0,-361.018397,-326.731686,-3.610184,-3.267317,1906.124816,1931.467683


### Optional: Plot Forward Outright Prices graph

Create a Forward Outright Prices dataframe used to plot a graph

In [27]:
fwd_outright = pd.DataFrame()
fwd_outright['DATE'] = df_fwd['END']
fwd_outright['BID_FWDOUTRIGHT'] = df_fwd['BID_FWDOUTRIGHT']
fwd_outright['ASK_FWDOUTRIGHT'] = df_fwd['ASK_FWDOUTRIGHT']
fwd_outright.index = fwd_outright['DATE']
del fwd_outright['DATE']
fwd_outright

,BID_FWDOUTRIGHT,ASK_FWDOUTRIGHT
DATE,,
2020-07-03,1909.798608,1934.814529
2020-07-06,1909.782706,1934.794647
2020-07-07,1909.715118,1934.719098
2020-07-13,1909.595825,1934.623686
2020-07-20,1909.456650,1934.512373
2020-07-27,1909.317474,1934.401059
2020-08-06,1909.118653,1934.242040
2020-09-08,1907.223568,1932.462084
2020-10-06,1906.124816,1931.467683


Plot BID and ASK Forward Outright Prices graph according to the dataframe

In [28]:
cf.set_config_file(offline=True)
mylayout = cf.Layout(title={
        'text': "Forward Outright of RIC " + metal + " based on spot price at " + metal_datetime + "(UTC)",
        'y':0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'})
fwd_outright.iplot(kind='line',layout=mylayout)

### Step 7: Close the RDP session

In [29]:
rdp.close_session()

# Summary

In this article, we have demonstrate how to calculate Forward Outright Price which its input can be retrived using RDP library for python. The article also show how to use Cufflinks which makes financial data visualization convenient. The RDP library can be used to retreive data for calculation any other models to serve more accurate or specific use cases.

# References

1. Refinitiv Data Platform Libraries, Refinitiv Developer Community, https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries

2. Quantitative Analytics - Dates and Calendars in API Playground, https://apidocs.refinitiv.com/Apps/ApiDocs#/details/L2RhdGEvcXVhbnRpdGF0aXZlLWFuYWx5dGljcy1kYXRlcy1hbmQtY2FsZW5kYXJzL2JldGEx/L2RhdGUtc2NoZWR1bGU=/POST/SWAGGER

3. "What Is Tenor?", in Tenor. Retrieved Jul 3, 2020 from https://www.investopedia.com/terms/t/tenor.asp

4. "What Is a Transaction Date?", in Transaction Date. Retrieved Jul 3, 2020, from https://www.investopedia.com/terms/t/transaction-date.asp
